### Import Modules

In [335]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit

matplotlib.rcParams['figure.figsize'] = [10, 10]

In [374]:
reload(bs)
reload(dl)
reload(fit)

<module 'fitter.fitter' from '../fitter\fitter.pyc'>

### Specify fit parameters

In [375]:
p_dict = {
    'order' : 1,
    'bs_N' : 20,  # if 0, use full list
    
    'use_prior' : True,
    'save_prior' : False,
    'abbrs' : ['a09m400', 'a12m130', 'a12m220', 'a12m220L', 
               'a12m220S', 'a12m310', 'a12m350', 'a12m400', 
               'a15m220', 'a15m310'],
    
    'make_plots' : True,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False
}

### Do fit

In [376]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior()

# Make bootstrapper
bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])


In [390]:
#bootstrapper.plot_fit_bar_graph()

In [385]:
bootstrapper.plot_parameters(xy_parameters=['Fpi', 'FK/Fpi'],
                    xlabel='$F_\pi$ (MeV)', color_parameter='a')

TypeError: float() argument must be a string or a number

In [ ]:
#    # xy_parameters is an array (eg, ['mpi', 'Fpi'])
    def plot_parameters(self, xy_parameters, color_parameter=None,
                        xfcn=None, xlabel=None, yfcn=None, ylabel=None, show_fit=True):
        if xlabel is None:
            xlabel = self._fmt_key_as_latex(xy_parameters[0])
        if ylabel is None:
            ylabel = self._fmt_key_as_latex(xy_parameters[1])

        if xfcn is None:
            xfcn = lambda x : 1 * x
        if yfcn is None:
            yfcn = lambda y : 1 * y

        plot_data = {}
        myfcn = [xfcn, yfcn]
        for j, parameter in enumerate(xy_parameters):
            if parameter in ['FK/Fpi', 'FK / Fpi']:
                plot_data[j] = {abbr :  myfcn[j](self.fit_data[abbr]['FK'] /self.fit_data[abbr]['Fpi']) for abbr in self.abbrs}
            elif parameter in ['mpi', 'mju', 'mru', 'mk', 'mrs', 'mss', 'FK', 'Fpi']:
                # Convert to physical units
                hbar_c = 197.327# used to convert to phys units
                plot_data[j] = {}
                for abbr in self.abbrs:
                    plot_data[j][abbr] = myfcn[j](self.fit_data[abbr][parameter] *hbar_c / (self.fit_data[abbr]['aw0'] /self.w0))
            else:
                plot_data[j] = {abbr :  myfcn[j](self.fit_data[abbr][parameter]) for abbr in self.abbrs}


        if color_parameter is None:
            color_parameter = 'a'

        if color_parameter in ['a']:
            color_data = {abbr : np.repeat(self.fit_data[abbr]['aw0'][0]/self.w0, self.bs_N).ravel() for abbr in self.abbrs}
        elif color_parameter in ['L']:
            color_data = {abbr : np.repeat(gv.mean(self.fit_data[abbr][color_parameter]), self.bs_N).ravel() for abbr in self.abbrs}
        elif color_parameter == 'MpiL':
            color_data = {abbr : gv.mean(self.fit_data[abbr][color_parameter]).ravel() for abbr in self.abbrs}
        else:
            color_data = {abbr : gv.mean(self.fit_data[abbr][color_parameter]).ravel() for abbr in self.abbrs}

        # Color by lattice spacing/length
        cmap = matplotlib.cm.get_cmap('rainbow')
        min_max = lambda x : [np.min(x), np.max(x)]
        minimum, maximum = min_max(np.concatenate([color_data[abbr] for abbr in self.abbrs]))
        norm = matplotlib.colors.Normalize(vmin=minimum, vmax=maximum)

        # Get scatter plot & color data
        x = np.zeros(self.bs_N * len(self.abbrs))
        y = np.zeros(self.bs_N * len(self.abbrs))
        z = np.zeros(self.bs_N * len(self.abbrs))
        for j, abbr in enumerate(self.abbrs):
            x[j*self.bs_N:(j+1)*self.bs_N] = gv.mean(plot_data[0][abbr])
            y[j*self.bs_N:(j+1)*self.bs_N] = gv.mean(plot_data[1][abbr])
            z[j*self.bs_N:(j+1)*self.bs_N] = color_data[abbr]

        sc = plt.scatter(x, y, c=z, vmin=minimum, vmax=maximum,
                         cmap=cmap, rasterized=True, marker=".", alpha=100.0/self.bs_N)

        if ((show_fit)
                and (xy_parameters[1] in ['FK/Fpi', 'FK / Fpi'])
                and (xy_parameters[0] in['mpi', 'mju', 'mru', 'mk', 'mrs', 'mss', 'FK', 'Fpi'])):

            plt.axvline(gv.mean(xfcn(self.get_phys_point_data(xy_parameters[0]))), label='Phys point')
            y_phys = self.get_phys_point_data('FK/Fpi')
            plt.errorbar(x=gv.mean(xfcn(self.get_phys_point_data(xy_parameters[0]))), xerr=0,
                         y=gv.mean(y_phys), yerr=gv.sdev(y_phys), label='Exp',
                        color='C0', marker='o', capsize=0.0, mec='white', ms=10.0, alpha=0.6,
                             ecolor='C1', elinewidth=10.0)

            colors = ['black', 'purple', 'green', 'red']
            for j, a in enumerate([0.0, 0.09, 0.12, 0.15]):
                minimum = np.nanmin([np.nanmin(
                    self.fit_data[abbr][xy_parameters[0]] *hbar_c / (self.fit_data[abbr]['aw0'] /self.w0)
                ) for abbr in self.abbrs])
                maximum = np.nanmax([np.nanmax(
                    self.fit_data[abbr][xy_parameters[0]] *hbar_c / (self.fit_data[abbr]['aw0'] /self.w0)
                ) for abbr in self.abbrs])
                delta = maximum - minimum

                x = np.linspace(np.max((minimum - 0.05*delta, 0)), maximum + 0.05*delta)


                # Get phys point data, substituting x-data and current 'a' in loop
                prepped_data = self.get_phys_point_data()
                prepped_data[xy_parameters[0]] = x
                prepped_data['aw0'] = a*self.w0

                y = self.fk_fpi_fit_fcn(fit_data=prepped_data)

                pm = lambda g, k : gv.mean(g) + k*gv.sdev(g)
                plt.plot(xfcn(x), pm(y, 0), '--', color=colors[j], label='$a=$%s (fm)'%(str(a)), rasterized=True)
                plt.fill_between(xfcn(x), pm(y, -1), pm(y, 1), alpha=0.20, color=colors[j], rasterized=True)

            plt.legend()

        # Plot labels
        plt.grid()
        plt.xlabel(xlabel, fontsize = 24)
        plt.ylabel(ylabel, fontsize = 24)

        # Format colorbar
        color_bar = plt.colorbar(sc)
        color_bar.set_alpha(0.8)
        color_bar.draw_all()
        color_bar.set_label(self._fmt_key_as_latex(color_parameter), fontsize = 24)

        # Set xlim, ylim -- only works if xy_parameters[i] is a vector, not a scalar
        min_max = lambda x : [np.min(x), np.max(x)]
        try:
            xmin, xmax = min_max(np.concatenate([gv.mean(plot_data[0][abbr]) for abbr in self.abbrs]))
            ymin, ymax = min_max(np.concatenate([gv.mean(plot_data[1][abbr]) for abbr in self.abbrs]))
            xdelta = xmax - xmin
            ydelta = ymax - ymin
            plt.xlim(xmin-0.05*xdelta, xmax+0.05*xdelta) #xmin-0.05*xdelta
            plt.ylim(ymin-0.05*ydelta, ymax+0.05*ydelta)
        except ValueError:
            pass

        fig = plt.gcf()
        plt.close()
        return fig
    
    squared = lambda x : x**2
    plot_parameters(bootstrapper, xy_parameters=['mpi', 'FK/Fpi'],
                    xfcn=squared, xlabel='$m_\pi^2$ (MeV)$^2$', color_parameter='a')

In [391]:
#bootstrapper._make_fit(0).show_plots()

In [389]:
#print bootstrapper._make_fit(0)

In [ ]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior()

# Make bootstrapper
bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

if p_dict['make_plots']:
    data_loader.save_plots(bootstrapper.make_plots(
        show_error_ellipses=p_dict['show_error_ellipses'], show_bootstrap_histograms=p_dict['show_bs_histograms']))
else:
    print bootstrapper
    
if p_dict['save_prior']:
    print 'Saving prior...'
    data_loader.save_prior(bootstrapper.create_prior_from_fit())
    
    
t1 = time.time()

print "\nTotal time (s): ", t1 - t0, "\n"

Making fits...
100% complete  Time (s):  2.39598703384
Getting bs fit parameters...
38% complete 